## Model Body Language 

Data Exploration - Thea 

In [107]:
!pip install mediapipe opencv-python

python(69878) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


## Initialize MediaPipe 

In [1]:
import cv2
import mediapipe as mp
import numpy as np

Initialize BlazePose MediaPipe

In [2]:
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose_model = mp_pose.Pose(
    static_image_mode=False,
    model_complexity=1,
    enable_segmentation=False,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

I0000 00:00:1748420889.623221 36351713 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M3


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1748420889.713520 36351886 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1748420889.725757 36351886 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Normalization

In [3]:
# --- Normalize utility ---
def normalize(value, min_val, max_val):
    return round(10 * np.clip((value - min_val) / (max_val - min_val), 0, 1), 1)

Scoring: openness 

In [4]:
def openness_score(landmarks):
    if landmarks[15].visibility > 0.65 and landmarks[16].visibility > 0.65:
        return abs(landmarks[15].x - landmarks[16].x)
    return None

In [5]:
def leaning_score(landmarks):
    visible = all(landmarks[i].visibility > 0.5 for i in [11, 12, 23, 24])
    if not visible:
        return None
    mid_shoulder_x = (landmarks[11].x + landmarks[12].x) / 2
    mid_hip_x = (landmarks[23].x + landmarks[24].x) / 2
    return mid_shoulder_x - mid_hip_x

In [6]:
def fidgeting_score(landmark_sequence):
    total = 0
    valid_pairs = 0
    for i in range(1, len(landmark_sequence)):
        for idx in [15, 16]:
            if landmark_sequence[i][idx].visibility > 0.4 and landmark_sequence[i-1][idx].visibility > 0.4:
                dx = landmark_sequence[i][idx].x - landmark_sequence[i-1][idx].x
                dy = landmark_sequence[i][idx].y - landmark_sequence[i-1][idx].y
                dist = np.sqrt(dx**2 + dy**2)
                total += dist
                valid_pairs += 1
    return total if valid_pairs > 0 else None

Count hand gestures: need to modify a bit to fit the length of video


In [7]:
def count_hand_gestures(landmark_sequence, threshold=0.05):
    count = 0
    for i in range(1, len(landmark_sequence)):
        for idx in [15, 16]:  # Left and right wrists
            if landmark_sequence[i][idx].visibility > 0.5 and landmark_sequence[i-1][idx].visibility > 0.5:
                dx = landmark_sequence[i][idx].x - landmark_sequence[i-1][idx].x
                dy = landmark_sequence[i][idx].y - landmark_sequence[i-1][idx].y
                dist = np.sqrt(dx**2 + dy**2)
                if dist > threshold:
                    count += 1
    return count

In [8]:
def posture_score(landmarks):
    # Shoulders aligned and stacked over hips
    if not all(landmarks[i].visibility > 0.5 for i in [11, 12, 23, 24]):
        return None
    shoulder_diff_y = abs(landmarks[11].y - landmarks[12].y)
    torso_angle = abs(((landmarks[11].x + landmarks[12].x)/2) - ((landmarks[23].x + landmarks[24].x)/2))
    return (1 - (shoulder_diff_y + torso_angle))  # closer to 1 is better

In [9]:
def space_occupation_score(landmark_sequence):
    total_motion = 0
    valid_frames = 0

    for i in range(1, len(landmark_sequence)):
        for idx in [15, 16]:  # Wrists
            lm1 = landmark_sequence[i][idx]
            lm0 = landmark_sequence[i - 1][idx]
            if lm1.visibility > 0.5 and lm0.visibility > 0.5:
                dx = lm1.x - lm0.x
                dy = lm1.y - lm0.y
                dist = np.sqrt(dx**2 + dy**2)
                total_motion += dist
                valid_frames += 1

    return total_motion / valid_frames if valid_frames > 0 else None


In [10]:
def fidgeting_score2(landmark_sequence, jitter_threshold=0.01):
    jitter_count = 0
    valid_frames = 0

    for i in range(1, len(landmark_sequence)):
        for idx in [15, 16]:  # wrists
            lm1 = landmark_sequence[i][idx]
            lm0 = landmark_sequence[i - 1][idx]
            if lm1.visibility > 0.5 and lm0.visibility > 0.5:
                dx = lm1.x - lm0.x
                dy = lm1.y - lm0.y
                dist = np.sqrt(dx**2 + dy**2)

                # Only count very small movements (jitter)
                if dist < jitter_threshold:
                    jitter_count += 1

                valid_frames += 1

    # Return % of frames that had tiny twitchy motion
    return (jitter_count / valid_frames) if valid_frames > 0 else None

In [11]:
def fidgeting_score3(landmark_sequence, jitter_threshold=0.01):
    jitter_ratio = 0
    movement_total = 0
    valid = 0

    for i in range(1, len(landmark_sequence)):
        for idx in [15, 16]:
            lm1 = landmark_sequence[i][idx]
            lm0 = landmark_sequence[i - 1][idx]
            if lm1.visibility > 0.5 and lm0.visibility > 0.5:
                dx = lm1.x - lm0.x
                dy = lm1.y - lm0.y
                dist = np.sqrt(dx**2 + dy**2)
                if dist < jitter_threshold:
                    jitter_ratio += 1
                movement_total += dist
                valid += 1

    if valid == 0:
        return None

    # Mix of average motion and jitter frequency
    return (jitter_ratio / valid) + 0.3 * (movement_total / valid)

In [18]:
# --- Combine Scores with Confidence Filtering ---
def interpret_pose(landmark_sequence):
    if not landmark_sequence:
        return None

    current = landmark_sequence[-1]
    scores = {}
    valid_scores = []
    space_score = space_occupation_score(landmark_sequence)
    if space_score is not None:
        space_norm = normalize(space_score, 0.001, 0.02)  # Tune thresholds
        space_norm=10*space_norm
        scores["Space Occupation"] = space_norm
        valid_scores.append(space_norm)


    lean_score = leaning_score(current)
    if lean_score is not None:
        lean_norm = 10 - normalize(abs(lean_score), 0, 0.2)
        lean_norm=10*lean_norm
        scores["Leaning Stability"] = lean_norm

        valid_scores.append(lean_norm)

    fidg_score = fidgeting_score3(landmark_sequence)
    if fidg_score is not None:
        fidg_norm = 10 - normalize(fidg_score, 0.005, 0.05)
        fidg_norm=10*fidg_norm
        scores["Composure Score"] = fidg_norm

        valid_scores.append(fidg_norm)

    gestures = count_hand_gestures(landmark_sequence)
    gesture_norm = normalize(gestures, 1, 20)  # higher is more expressive
    gesture_norm=10*gesture_norm
    scores["Hand Gesture Activity"] = gesture_norm
    if gesture_norm < 3:

        valid_scores.append(gesture_norm)
        valid_scores.append(gesture_norm)  # counts double
    else:
        valid_scores.append(gesture_norm)


    posture = posture_score(current)
    if posture is not None:
        posture_norm = normalize(posture, 0.7, 1.0)
        posture_norm=10*posture_norm
        scores["Posture Alignment"] = posture_norm

        valid_scores.append(posture_norm)

    if valid_scores:
        low_scores = [s for s in valid_scores if s <= 5 and s!=0]
    if len(low_scores) >= 1:
        max_low = max(low_scores)
        valid_scores.append(max_low)  # count max low score twice
    if valid_scores:
        scores["Overall Body Language Score"] = round(sum(valid_scores) / len(valid_scores), 1)

    return scores if scores else None

In [19]:
# --- Analyze a Specific Video File ---
def analyze_video(video_path):
    cap = cv2.VideoCapture(video_path)
    landmark_sequence = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose_model.process(image_rgb)

        if results.pose_landmarks:
            landmark_sequence.append(results.pose_landmarks.landmark)

    cap.release()
    return interpret_pose(landmark_sequence)
video_path = "/Users/theaalfon/Desktop/TestVid.MOV"
analyze_video(video_path)

{'Space Occupation': 50.0,
 'Composure Score': 0.0,
 'Hand Gesture Activity': 100.0,
 'Overall Body Language Score': 50.0}

In [22]:
video_path = "/Users/theaalfon/Desktop/TestVid.MOV"

def analyze_score(video_path):
    scores = analyze_video(video_path)
    print("\nPose Analysis Results:")
    if scores:
        for key, value in scores.items():
            print(f"{key}: {value}%")
    else:
        print("No valid pose detected in the video.")

In [23]:
video_path = "/Users/theaalfon/Downloads/tedtalk.mov"
analyze_score(video_path)


Pose Analysis Results:
Space Occupation: 100.0%
Leaning Stability: 71.0%
Composure Score: 0.0%
Hand Gesture Activity: 100.0%
Posture Alignment: 68.0%
Overall Body Language Score: 67.8%


In [24]:
video_path ="/Users/theaalfon/Downloads/comedian.MP4"
analyze_score(video_path)


Pose Analysis Results:
Space Occupation: 67.0%
Composure Score: 0.0%
Hand Gesture Activity: 100.0%
Overall Body Language Score: 55.7%


In [25]:
video_path ="/Users/theaalfon/Downloads/comedian2.MP4"
analyze_score(video_path)


Pose Analysis Results:
Space Occupation: 100.0%
Leaning Stability: 95.0%
Composure Score: 0.0%
Hand Gesture Activity: 100.0%
Posture Alignment: 72.0%
Overall Body Language Score: 73.4%


In [26]:
video_path ="/Users/theaalfon/Downloads/comedian3.MP4"
analyze_score(video_path)


Pose Analysis Results:
Space Occupation: 100.0%
Leaning Stability: 99.0%
Composure Score: 0.0%
Hand Gesture Activity: 100.0%
Posture Alignment: 95.0%
Overall Body Language Score: 78.8%


In [27]:
video_path ="/Users/theaalfon/Downloads/anxious.MP4"
analyze_score(video_path)


Pose Analysis Results:
Space Occupation: 94.0%
Leaning Stability: 88.0%
Composure Score: 0.0%
Hand Gesture Activity: 16.0%
Posture Alignment: 91.0%
Overall Body Language Score: 57.8%


In [28]:
video_path ="/Users/theaalfon/Downloads/nervous.mp4"
analyze_score(video_path)


Pose Analysis Results:
Space Occupation: 9.0%
Composure Score: 0.0%
Hand Gesture Activity: 89.0%
Overall Body Language Score: 32.7%


In [29]:
video_path ="/Users/theaalfon/Downloads/anxious2.MP4"
analyze_score(video_path)


Pose Analysis Results:
Space Occupation: 31.0%
Leaning Stability: 43.0%
Composure Score: 0.0%
Hand Gesture Activity: 100.0%
Posture Alignment: 31.0%
Overall Body Language Score: 41.0%


In [30]:
video_path ="/Users/theaalfon/Downloads/default.MOV"
analyze_score(video_path)


Pose Analysis Results:
No valid pose detected in the video.


In [130]:
def annotate_video(video_path, output_path="annotated_output.mp4", display=False):
    cap = cv2.VideoCapture(video_path)

    # Get video properties
    fps = cap.get(cv2.CAP_PROP_FPS)
    width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Output writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose_model.process(image_rgb)

        if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                frame,
                results.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
                connection_drawing_spec=mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2, circle_radius=1)
            )

        out.write(frame)

        if display:
            cv2.imshow('Pose Detection', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    cap.release()
    out.release()
    if display:
        cv2.destroyAllWindows()

    print(f"\n✅ Annotated video saved to: {output_path}")


In [131]:
# Run pose landmark overlay and export
annotate_video("/Users/theaalfon/Downloads/anxious2.MP4", display=True)


✅ Annotated video saved to: annotated_output.mp4
